In [6]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os, gc
import pickle
import datetime, time
from tqdm import tqdm_notebook as tqdm


from sklearn import preprocessing

%matplotlib inline

In [7]:
path = '/Users/kanoumotoharu/Downloads/m5-forecasting-accuracy/'

In [8]:
os.listdir(path)

['train_df2.csv',
 'calendar.csv',
 'sell_prices.csv',
 'sales_train_validation.csv',
 'sample_submission.csv']

In [9]:
%%time
train_df = pd.read_csv(path+'sales_train_validation.csv')
calendar_df = pd.read_csv(path+'calendar.csv')
sell_prices_df = pd.read_csv(path+'sell_prices.csv')
sample_submission_df = pd.read_csv(path+'sample_submission.csv')

KeyboardInterrupt: 

In [10]:
train_df.set_index('id', inplace=True)
train_df.head()

item_id    dept_id   cat_id store_id  \
id                                                                          
HOBBIES_1_001_CA_1_validation  HOBBIES_1_001  HOBBIES_1  HOBBIES     CA_1   
HOBBIES_1_002_CA_1_validation  HOBBIES_1_002  HOBBIES_1  HOBBIES     CA_1   
HOBBIES_1_003_CA_1_validation  HOBBIES_1_003  HOBBIES_1  HOBBIES     CA_1   
HOBBIES_1_004_CA_1_validation  HOBBIES_1_004  HOBBIES_1  HOBBIES     CA_1   
HOBBIES_1_005_CA_1_validation  HOBBIES_1_005  HOBBIES_1  HOBBIES     CA_1   

                              state_id  d_1  d_2  d_3  d_4  d_5   ...    \
id                                                                ...     
HOBBIES_1_001_CA_1_validation       CA    0    0    0    0    0   ...     
HOBBIES_1_002_CA_1_validation       CA    0    0    0    0    0   ...     
HOBBIES_1_003_CA_1_validation       CA    0    0    0    0    0   ...     
HOBBIES_1_004_CA_1_validation       CA    0    0    0    0    0   ...     
HOBBIES_1_005_CA_1_validation       CA    0    0    0    0    0   ...     

                               d_1904  d_1905  d_1906  d_1907  d_1908  d_1909  \
id                                                                              
HOBBIES_1_001_CA_1_validation       1       3       0       1       1       1   
HOBBIES_1_002_CA_1_validation       0       0       0       0       0       1   
HOBBIES_1_003_CA_1_validation       2       1       2       1       1       1   
HOBBIES_1_004_CA_1_validation       1       0       5       4       1       0   
HOBBIES_1_005_CA_1_validation       2       1       1       0       1       1   

                               d_1910  d_1911  d_1912  d_1913  
id                                                             
HOBBIES_1_001_CA_1_validation       3       0       1       1  
HOBBIES_1_002_CA_1_validation       0       0       0       0  
HOBBIES_1_003_CA_1_validation       0       1       1       1  
HOBBIES_1_004_CA_1_validation       1       3       7       2  
HOBBIES_1_005_CA_1_validation       2       2       2       4  

[5 rows x 1918 columns]

In [11]:
calendar_df.head()

date  wm_yr_wk    weekday  wday  month  year    d event_name_1  \
0  2011-01-29     11101   Saturday     1      1  2011  d_1          NaN   
1  2011-01-30     11101     Sunday     2      1  2011  d_2          NaN   
2  2011-01-31     11101     Monday     3      1  2011  d_3          NaN   
3  2011-02-01     11101    Tuesday     4      2  2011  d_4          NaN   
4  2011-02-02     11101  Wednesday     5      2  2011  d_5          NaN   

  event_type_1 event_name_2 event_type_2  snap_CA  snap_TX  snap_WI  
0          NaN          NaN          NaN        0        0        0  
1          NaN          NaN          NaN        0        0        0  
2          NaN          NaN          NaN        0        0        0  
3          NaN          NaN          NaN        1        1        0  
4          NaN          NaN          NaN        1        0        1

In [12]:
sell_prices_df['id'] = sell_prices_df.item_id+'_'+sell_prices_df.store_id+'_validation'
sell_prices_df.head()

NameError: name 'sell_prices_df' is not defined

In [ ]:
sell_prices_data = sell_prices_df.groupby(['id'])['wm_yr_wk'].unique()

In [ ]:
sell_prices_data = sell_prices_data.apply(lambda x: np.sort(x))

In [ ]:
sell_prices_data = sell_prices_data.apply(lambda x: x[0])

In [ ]:
wm_to_d = calendar_df[~calendar_df.wm_yr_wk.duplicated()].set_index('wm_yr_wk')['d'].to_dict()

In [ ]:
sell_prices_data = sell_prices_data.map(wm_to_d)

In [ ]:
sell_prices_data = sell_prices_data.apply(lambda x: [f'd_{i}' for i in range(1,int(x[2:]))] if int(x[2:])>1 else np.nan)

In [ ]:
sell_prices_data = sell_prices_data[sell_prices_data.notna()]# = sell_prices_data

In [ ]:
sell_prices_data = pd.concat([
    sell_prices_data,
    sell_prices_data.apply(lambda x: len(x))
    
], axis=1)

In [ ]:
sell_prices_data.columns = ['wm_yr_wk', 'len']
sell_prices_data.reset_index(inplace=True)
sell_prices_data.head()

In [ ]:
sell_prices_data = sell_prices_data.groupby(['len'])['id'].unique()

In [ ]:
sell_prices_data

In [ ]:
for _len, cols in tqdm(sell_prices_data.iteritems(), total=len(sell_prices_data)):
    train_df.loc[cols, [f'd_{i+1}' for i in range(_len)]]=np.nan
    #print(cols, [f'd_{i+1}' for i in range(_len)])

In [ ]:
train_df.head()

In [ ]:
train_df.to_csv(path+'train_df2.csv')

In [9]:
d_cols = [f'd_{i}' for i in range(1,1914)]

In [ ]:
sell_prices_df.groupby([''])

In [10]:
data = train_df.set_index('id')[d_cols].T
data.reset_index(drop=True, inplace=True)

In [11]:
calendar_df.date = pd.to_datetime(calendar_df.date)

In [12]:
calendar_df2 = calendar_df[:1913]

In [13]:
data = pd.concat([
    calendar_df2[['date', 'weekday']],
    data
],axis=1)